In [5]:
import pandas as pd
import numpy as np
import random
from quantnote import BacktestEngine, Strategy

In [6]:
db_path = '/Users/j.nahmgoong/Documents/git/DB/MainDB.db'

engine = BacktestEngine(db_path)

Loading DB...
DB loaded in 44.78 seconds
[initialize] is executed in 51.18 seconds


In [7]:
engine.show_sample_strategy()

class myStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def compute_target(self, universe_list):
        target_weight = { }
        for ticker in universe_list:
            target_weight[ticker] = 1
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            assert False



In [8]:
non_cap_norm = ['tbvps',
'sps',
'ros',
'roe',
'roa',
'ps1',
'ps',
'price',
'prefdivis',
'pe1',
'pe',
'pb',
'payoutratio',
'fcfps',
'epsdil',
'eps',
'dps',
'divyield',
'de',
'currentratio',
'bvps',
'marketcap']

In [12]:
class myStrategy(Strategy):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        print(self.config)

    def compute_target(self, universe_list):
        target_weight = { }
        
        input1 = self.config['input1']
        input2 = self.config['input2']
        unary = self.config['unary']
        binary = self.config['binary']
        unary_out = self.config['unary_out']
        
        factor_series_1 = self.compute_factor_series(universe_list, input1, type=unary)
        factor_series_2 = self.compute_factor_series(universe_list, input2, type=unary)
        
        if binary == 'add':
            factor_series = factor_series_1 + factor_series_2
        elif binary == 'sub':
            factor_series = factor_series_1 - factor_series_2
        elif binary == 'mul':
            factor_series = factor_series_1 * factor_series_2
        elif binary == 'div':
            factor_series = factor_series_1 / factor_series_2
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        assert len(factor_series) > 0.5*len(universe_list)
        
        if unary_out == 'raw':
            pass
        elif unary_out == 'rank':
            factor_series = factor_series.rank(ascending=False)
        elif unary_out == 'zscore':
            factor_series = (factor_series-factor_series.mean())/factor_series.std()
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        top = int(len(factor_series.index)/5)
        
        for ticker in factor_series.index:
            weight = factor_series[ticker]
            if weight > 0:
                target_weight[ticker] = weight
            
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            if ftype in non_cap_norm:
                x = self.get_value('fundamentals',ticker,ftype)
            else:
                x = self.get_value('fundamentals',ticker,ftype)/self.get_value('fundamentals',ticker,'marketcap')
            return x



In [13]:
input_list = list(engine.cache['fundamentals']['AAPL'].columns[5:])
unary_list = ['raw', 'rank', 'zscore']
binary_list = ['add', 'sub', 'mul', 'div']

results = []

config_set = []


len(input_list)

105

In [ ]:
idx = 1
while True:
    x1, x2 = random.sample(input_list,2)
    unary = random.sample(unary_list,1)[0]
    binary = random.sample(binary_list,1)[0]
    unary_out = random.sample(unary_list,1)[0]
    
    config = {
    'input1':x1,
    'input2':x2,
    'unary':unary,
    'binary':binary,
    'unary_out':unary_out}
    
    if config in config_set:
        pass
    else:
        config_set.append(config)
    
    print('\n')
    print('Computing {}th factor'.format(idx))
    mys = myStrategy(config)
    try:
        engine.run_backtest(mys, '2016-01-01','2021-08-31',period='Q')
        engine.stat(bench=None)
        print(engine.stat_dic)

        results.append([engine.stat_dic['sharpe'],engine.stat_dic['cagr'],config])
        idx += 1
    except:
        print('Passed!')
        
    results.sort(key = lambda x:-x[0])
    print('Current best sharpe')
    print(results[0])




Computing 1th factor
{'input1': 'intangibles', 'input2': 'marketcap', 'unary': 'raw', 'binary': 'sub', 'unary_out': 'rank'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.127 / time elapsed:137.2 ===
[run_backtest] is executed in 137.93 seconds
{'cagr': 0.13315838781467149, 'vol': 0.18311105014551962, 'sharpe': 0.7272001755702324}
Current best sharpe
[0.7272001755702324, 0.13315838781467149, {'input1': 'intangibles', 'input2': 'marketcap', 'unary': 'raw', 'binary': 'sub', 'unary_out': 'rank'}]


Computing 2th factor
{'input1': 'ros', 'input2': 'currentratio', 'unary': 'raw', 'binary': 'div', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.969 / time elapsed:113.5 ===
[run_backtest] is executed in 114.18 seconds
{'cagr': 0.11955160949990569, 'vol': 0.17772812420460818, 'sharpe': 0.6726656798688361}
Current best sharpe
[0.7272001755702324, 0.13315838781

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.281 / time elapsed:133.0 ===/ time elapsed:133.2 ===
[run_backtest] is executed in 134.07 seconds
{'cagr': 0.1454424178790198, 'vol': 0.1860121722598068, 'sharpe': 0.7818973140955398}
Current best sharpe
[0.9023788582842298, 0.16911432629051273, {'input1': 'taxliabilities', 'input2': 'assetsnc', 'unary': 'raw', 'binary': 'mul', 'unary_out': 'rank'}]


Computing 17th factor
{'input1': 'divyield', 'input2': 'opex', 'unary': 'zscore', 'binary': 'add', 'unary_out': 'rank'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.403 / time elapsed:134.0 ===
[run_backtest] is executed in 134.73 seconds
{'cagr': 0.1548704870083921, 'vol': 0.18945883349659198, 'sharpe': 0.817436084399717}
Current best sharpe
[0.9023788582842298, 0.16911432629051273, {'input1': 'taxliabilities', 'input2': 'assetsnc', 'unary': 'raw', 'binary': 'mul', 'unary_out